### Импорт библиотек

In [52]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import json
import tqdm
import time

import pandas as pd

### Тесты загрузки

In [ ]:
# Опции для браузера (например, без GUI, если нужно)
chrome_options = Options()
chrome_options.add_argument("--headless")  # если не нужен графический интерфейс
#chrome_options.add_argument("--no-sandbox")
#chrome_options.add_argument("--disable-dev-shm-usage")

# Путь к chromedriver и инициализация сервиса
service = Service(executable_path="/usr/bin/chromedriver")
# Создаем драйвер с опциями
browser = webdriver.Chrome(service=service, options=chrome_options)
# Задаем заголовки User-Agent через CDP (Chrome DevTools Protocol)
browser.execute_cdp_cmd('Network.setUserAgentOverride', {
    "userAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 YaBrowser/25.6.0.0 Safari/537.36"
})

# Открываем страницу
url_main = 'https://kinogo.org/serials/'
browser.get(url_main)
# Получаем HTML страницы
src = browser.page_source
# Вывод или сохранение
print(src)

# Закрываем браузер
browser.quit()

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ru" lang="ru" prefix="og: http://ogp.me/ns# video: http://ogp.me/ns/video# ya: http://webmaster.yandex.ru/vocabularies/"><head>
    <title>Сериалы смотреть онлайн бесплатно в хорошем качестве на киного</title>
<meta name="charset" content="utf-8">
<meta name="title" content="Сериалы смотреть онлайн бесплатно в хорошем качестве на киного">
<meta name="description" content="Сериалы – самая большая кино-база мира кино. Смотреть Сериалы онлайн бесплатно на КиноГо">
<meta name="keywords" content="Сериалы, смотреть, онлайн, бесплатно, киного">
<meta name="generator" content="DataLife Engine (https://dle-news.ru)">
<link rel="canonical" href="https://kinogo.org/serials/">
<link rel="search" type="application/opensearchdescription+xml" title="KinoGo.org" href="https://kinogo.org/index.php?do=opensearch">
<link rel="preconnect" href="https://kinogo.org/" fetchpriority="high">
<meta property="twitter:card" content="summary">
<meta property="tw

In [13]:
# сохраним стартовую страницу
with open("kinogo_start_page.html", 'w') as file:
    file.write(src)

In [14]:
# продолжим работу с сохраненной страницей
with open("kinogo_start_page.html") as file:
    src = file.read()

In [15]:
soup = BeautifulSoup(src, "lxml")

In [70]:
# Название, ссылка на фильм
all_info_dict = {}
all_serials_hrefs = soup.find_all(class_='hTitle')
for item in all_serials_hrefs:
    a_tag = item.find('a')
    if a_tag:
        title = a_tag.get('title')  # например, "Подноготная (2025)"
        if title:
            # Уберем год и скобки, оставим только название
            item_text = title.split('(')[0].strip()  # "Подноготная"
        else:
            # Если title отсутствует, берем текст ссылки целиком
            item_text = a_tag.text.strip()
        item_href = a_tag.get('href')
        print(f'{item_text}: {item_href}')
        all_info_dict[item_text] = item_href


Струны души: https://kinogo.org/125122-struny-dushi.html
Любовь и гордость: https://kinogo.org/121822-ljubov-i-gordost.html
Док: https://kinogo.org/119089-dok.html
Убийство в маленьком городке: https://kinogo.org/115905-ubijstvo-v-malenkom-gorodke.html
Большой потенциал: https://kinogo.org/115729-bolshoj-potencial.html
Медленные лошади: https://kinogo.org/6694-medlennye-loshadi.html
Утреннее шоу: https://kinogo.org/8272-utrennee-shou.html
Бар «Один звонок»: https://kinogo.org/126522-bar-odin-zvonok.html
Морская полиция: Тони и Зива: https://kinogo.org/126003-morskaja-policija-toni-i-ziva.html
Платонические отношения: https://kinogo.org/4745-platonicheskie-otnoshenija.html


In [122]:
# Год выпуска
blocks = soup.find_all(class_="shortStory")
final_dict = []
for block in blocks:
    year_title_tag = block.find('b', string='Год выпуска:')
    if year_title_tag:
        year_tags = year_title_tag.find_next_siblings('a')
        year_texts = [tag.get_text() for tag in year_tags]
        year_str = ", ".join(year_texts)
        print(year_str)
    #     final_dict['country'].append(year_str)
    # else:
    #     final_dict['country'].append(None)

2025
2025
2025
2025
2025
2023
2024
2024
2025
2025


In [121]:
# URL постера
final_dict = {}
for block in blocks:
    image_tag = block.find("div", class_="sPoster").find("img").get('data-src')
    if image_tag:
        image_url = 'https://kinogo.org' + image_tag
        print(image_url)
    #     final_dict['image_movie'].append(image_url)
    # else:
    #     final_dict['image_movie'].append(None)


https://kinogo.org/uploads/mini/short/09c/9fe24f15918df18f9fa1c495a6ad9.webp
https://kinogo.org/uploads/mini/short/3b0/5618c9f935bbec17a860e7b2c64c7.webp
https://kinogo.org/uploads/mini/short/13e/6ddd07fd7f4e5d4fe7237579327b8.webp
https://kinogo.org/uploads/mini/short/732/1e42b0a05e294831167520b414305.webp
https://kinogo.org/uploads/mini/short/ca3/fd8c8f3d2af64342bc1c16a46cac5.webp
https://kinogo.org/uploads/mini/short/e5b/af8401186e22b90bf0fe7d6489d4b.webp
https://kinogo.org/uploads/mini/short/088/b1ed8d37d7e8dd9fb946911577897.webp
https://kinogo.org/uploads/mini/short/7e1/d0da09991154ead165d53c9c88c20.webp
https://kinogo.org/uploads/mini/short/b17/bd9a43b740dfe558d0562f3c28c12.webp
https://kinogo.org/uploads/mini/short/a48/d9cf56d2c95f04214c8810e3f424a.webp


In [141]:
for block in blocks:
    rating_tag = block.find("span", class_="ratingStats")
    if rating_tag:
        rating = rating_tag.get_text(strip=True).split()[0]
        print(rating)
        final_dict['rating_votes'].append(rating)
    else:
        final_dict['rating_votes'].append(None)

4.2/5
3.2/5
3.1/5
3.4/5
3.6/5
3.6/5
3.7/5
3.5/5
4.4/5
4.2/5


In [ ]:
 # 10. Рейтинг Кинопоиска
for block in blocks:
    rating_kp_tag = block.find("span", class_="fullKP")
    if rating_kp_tag:
        rating_kp = rating_kp_tag.get_text(strip=True).split(':')[1]
        print(rating_kp)
        final_dict['film_rating_kp'].append(rating_kp_tag)
    else:
        final_dict['film_rating_kp'].append(None)

Неизвестно
Неизвестно
7.9
7.6
Неизвестно
7.3
7.0
Неизвестно
Неизвестно
Неизвестно


In [147]:
for block in blocks:
    rating_kp_tag = block.find("span", class_="fullKP")
    if rating_kp_tag:
        rating_kp = rating_kp_tag.get_text(strip=True).split(':')[1]
        print(rating_kp)
    #     final_dict['film_rating_kp'].append(rating_kp_tag)
    # else:
    #     final_dict['film_rating_kp'].append(None)

Неизвестно
Неизвестно
7.9
7.6
Неизвестно
7.3
7.0
Неизвестно
Неизвестно
Неизвестно


In [ ]:
# 11. Рейтинг IMDb
for block in blocks:
    rating_imdb_tag = block.find("span", class_="fQuality")
    if rating_imdb_tag:
        rating_imdb = rating_imdb_tag.get_text(strip=True).split(':')[1]
        print(rating_imdb)
    #     final_dict['film_rating_imdb'].append(rating_imdb)
    # else:
    #     final_dict['film_rating_imdb'].append(None)

Неизвестно
Неизвестно
Неизвестно
Неизвестно
Неизвестно
7.6
6.5
7.9
Неизвестно
Неизвестно


In [155]:
final_dict['url_movie'][:2]

['https://kinogo.org/126489-pin-kod.html',
 'https://kinogo.org/126326-rejs-314.html']

In [156]:
for url in final_dict['url_movie'][:2]:
    if url:
        try:
            browser.get(url)
            # Явное ожидание загрузки через WebDriverWait лучше, но для простоты:
            time.sleep(2)  # уменьшено с 3-5 сек
            movie_soup = BeautifulSoup(browser.page_source, "lxml")
            desc_block = movie_soup.find("div", class_="filmDescription")
            print(desc_block)
            if desc_block:
                full_desc = desc_block.get_text(strip=True)
            else:
                full_desc = None
        except Exception as e:
            print(f"Ошибка при загрузке {url}: {e}")
            full_desc = None
    else:
        full_desc = None

    #descriptions.append(full_desc)


Ошибка при загрузке https://kinogo.org/126489-pin-kod.html: HTTPConnectionPool(host='localhost', port=49875): Max retries exceeded with url: /session/2577eb5fab928523936854e9723ac4d8/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78f2e6597a00>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ошибка при загрузке https://kinogo.org/126326-rejs-314.html: HTTPConnectionPool(host='localhost', port=49875): Max retries exceeded with url: /session/2577eb5fab928523936854e9723ac4d8/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78f2e6596da0>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [125]:
# 8. Описание
for block in blocks:
    description_title_tag = block.find("p", class_="excerpt")
    if description_title_tag:
        print(description_title_tag.get_text(strip=True))
        print()
    #     final_dict['description'].append(description_title_tag.get_text(strip=True))
    # else:
    #     final_dict['description'].append(None)

Максим Захаров, майор полиции, сталкивается с личной трагедией, когда его дочь становится жертвой аферистов и впадает в кому после попытки самоубийства. Решив взять дело в свои руки, Захаров переходит в отдел, занимающийся борьбой с мошенничеством. В процессе расследования он встречает Евгению, работницу банка, и между ними...

Иван Никогда, бывший чиновник с коррупционным прошлым, вынужден скрываться от правосудия. Под вымышленной личностью, он оказывается в роли пилота на внутреннем авиарейсе из Москвы во Владивосток. Его планы лететь незамеченным рушатся, когда настоящий пилот неожиданно умирает, оставляя Ивана управлять самолетом, хотя тот...

Шашлычник Паша из Челябинска вместе с отцом и сыном Костей отправляются в поездку на юг страны, чтобы принять в наследство свою долю в винодельне. Вторая половина этого предприятия принадлежит Сергею Гордееву — тонкому ценителю и профессионалу своего дела, а также его дочери Оксане и внучке Миле. Двум семьям предстоит научиться...

Двое предс

In [ ]:
# словарь для записи результатов
final_dict = {
    'title': [],
    'out_year': [],
    'genres': [],
    'country': [],
    'duration': [],
    'description': [],
    'url_movie': [],
    'image_movie': [],
    'rating_votes': [],
    'film_rating_kp': [],
    'film_rating_imdb': []
}
# Для бд из 5000 сериалов понадобится с 1 по 500 стр (10 сериалов на каждой странице)
page_min = 1
page_max = 500
my_range = range(page_min, page_max+1)
# Список URL со всеми странциами
urls = [f'{url_main}/page/{i+1}/' for i in range(500)]  # Список URL

### Основной парсинг

In [164]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

# --- Настройки Selenium ---
chrome_options = Options()
chrome_options.add_argument("--headless")  # если не нужен графический интерфейс
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")

# Путь к chromedriver и инициализация сервиса
service = Service(executable_path="/usr/bin/chromedriver")
# Создаем драйвер с опциями
browser = webdriver.Chrome(service=service, options=chrome_options)
# Маскировка под браузер
browser.execute_cdp_cmd('Network.setUserAgentOverride', {
    "userAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 YaBrowser/25.6.0.0 Safari/537.36"
})

# --- Основные настройки ---
url_main = 'https://kinogo.org/serials/'
page_min = 1
page_max = 10
my_range = range(page_min, page_max+1)

# --- Словарь результатов ---
final_dict = {
    'title': [],
    'out_year': [],
    'genres': [],
    'country': [],
    'duration': [],
    'description': [],
    'url_movie': [],
    'image_movie': [],
    'rating_votes': [],
    'film_rating_kp': [],
    'film_rating_imdb': []
}

# --- Цикл по страницам ---
for page in my_range:
    start_time = time.time()
    url = f"{url_main}page/{page}/"
    print(f"🔎 Загружаем страницу {page}: {url}")

    browser.get(url)
    time.sleep(5)  # ждём прогрузку
    soup = BeautifulSoup(browser.page_source, "lxml")

    # 0. Находим все блоки сериалов
    blocks = soup.find_all(class_="shortStory")

    # 1. Название
    final_dict['title'].extend([
        block.find("h2").get_text(strip=True).split('(')[0].strip() if block.find("h2") else None
        for block in blocks
    ])

    # 2. Год выхода
    for block in blocks:
        year_title_tag = block.find('b', string='Год выпуска:')
        if year_title_tag:
            year_tags = year_title_tag.find_next_siblings('a')
            year_texts = [tag.get_text() for tag in year_tags]
            year_str = ", ".join(year_texts)
            final_dict['out_year'].append(year_str)
        else:
            final_dict['out_year'].append(None)

    # 3. URL фильма
    final_dict['url_movie'].extend([
        block.find("a").get("href") if block.find("a") else None
        for block in blocks
    ])

    # 4. URL постера
    for block in blocks:
        image_tag = block.find("div", class_="sPoster").find("img").get('data-src')
        if image_tag:
            image_url = 'https://kinogo.org' + image_tag
            final_dict['image_movie'].append(image_url)
        else:
            final_dict['image_movie'].append(None)

    # 5. Жанры
    for block in blocks:
        genre_title_tag = block.find('b', string='Жанр:')
        if genre_title_tag:
            genre_tags = genre_title_tag.find_next_siblings('a')
            movie_genres = [item.get_text() for item in genre_tags if item.get_text() not in ['Фильмы', 'Новинки']]
            final_dict['genres'].append(movie_genres)
        else:
            final_dict['genres'].append(None)

    # 6. Страна
    for block in blocks:
        country_title_tag = block.find('b', string='Страна:')
        if country_title_tag:
            country_tags = country_title_tag.find_next_siblings('a')
            movie_country = [item.get_text() for item in country_tags]
            final_dict['country'].append(movie_country)
        else:
            final_dict['country'].append(None)

    # 7. Продолжительность
    for block in blocks:
        duration_title_tag = block.find('b', string='Продолжительность:')
        if duration_title_tag:
            final_dict['duration'].append(duration_title_tag.next_sibling.strip())
        else:
            final_dict['duration'].append(None)

    # 8. Описание (краткое)
    short_descriptions = []
    for block in blocks:
        description_title_tag = block.find("p", class_="excerpt")
        if description_title_tag:
            short_des = description_title_tag.get_text(strip=True)
            final_dict['description'].append(short_des)
            short_descriptions.append(short_des)
        else:
            final_dict['description'].append(None)
            short_descriptions.append(None)


    # 9. Рейтинг сайта
    for block in blocks:
        rating_tag = block.find("span", class_="ratingStats")
        if rating_tag:
            rating = rating_tag.get_text(strip=True).split()[0]
            final_dict['rating_votes'].append(rating)
        else:
            final_dict['rating_votes'].append(None)

    # 10. Рейтинг Кинопоиска
    for block in blocks:
        rating_kp_tag = block.find("span", class_="fullKP")
        if rating_kp_tag:
            rating_kp = rating_kp_tag.get_text(strip=True).split(':')[1]
            final_dict['film_rating_kp'].append(rating_kp)
        else:
            final_dict['film_rating_kp'].append(None)

    # 11. Рейтинг IMDb
    for block in blocks:
        rating_imdb_tag = block.find("span", class_="fQuality")
        if rating_imdb_tag:
            rating_imdb = rating_imdb_tag.get_text(strip=True).split(':')[1]
            final_dict['film_rating_imdb'].append(rating_imdb)
        else:
            final_dict['film_rating_imdb'].append(None)

    end_time = time.time() - start_time
    print(f"✅ Страница {page} собрана за {end_time:.2f} сек")

    # Сохраняем каждую  страницу
    # if page % 100 == 0:
    current_df = pd.DataFrame(final_dict)
    current_df.to_csv('movies_dataset.csv', index=False)
    with open('movies_dataset.json', 'w', encoding='utf-8') as f:
        json.dump(final_dict, f, ensure_ascii=False, indent=4)

print("🎬 Основной парсинг завершён. Данные сохранены в movies_dataset.csv и в movies_dataset.json")
print("🔎 Загружаем подробные описания с отдельных страниц...")

# заменяем краткое описание на полное
descriptions = []
for idx, url in enumerate(final_dict['url_movie']):
    short_desc = short_descriptions[idx] if idx < len(short_descriptions) else None
    if url:
        try:
            browser.get(url)
            WebDriverWait(browser, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "filmDescription"))
            )
            movie_soup = BeautifulSoup(browser.page_source, "lxml")
            desc_block = movie_soup.find("div", class_="filmDescription")
            if desc_block:
                full_desc = desc_block.get_text(strip=True)
            else:
                full_desc = short_desc  # если нет полного описания - краткое
        except Exception as e:
            print(f"Ошибка при загрузке {url}: {e}")
            full_desc = short_desc  # при ошибке используем краткое описание
    else:
        full_desc = short_desc  # нет ссылки - краткое описание

    descriptions.append(full_desc)
final_dict['description'] = descriptions

# --- Финальное сохранение ---
current_df = pd.DataFrame(final_dict)
current_df.to_csv('movies_dataset_full.csv', index=False)
with open('movies_dataset_full.json', 'w', encoding='utf-8') as f:
    json.dump(final_dict, f, ensure_ascii=False, indent=4)

print("🎬 Парсинг завершён. Данные сохранены в movies_dataset_full.csv и movies_dataset_full.json")
print(f'Количество фильмов в базе: {current_df.shape[0]}')
current_df.head()

# --- Закрываем браузер ---
browser.quit()


🔎 Загружаем страницу 1: https://kinogo.org/serials/page/1/
✅ Страница 1 собрана за 10.45 сек
🔎 Загружаем страницу 2: https://kinogo.org/serials/page/2/
✅ Страница 2 собрана за 7.96 сек
🔎 Загружаем страницу 3: https://kinogo.org/serials/page/3/
✅ Страница 3 собрана за 6.24 сек
🔎 Загружаем страницу 4: https://kinogo.org/serials/page/4/
✅ Страница 4 собрана за 29.34 сек
🔎 Загружаем страницу 5: https://kinogo.org/serials/page/5/
✅ Страница 5 собрана за 27.41 сек
🔎 Загружаем страницу 6: https://kinogo.org/serials/page/6/
✅ Страница 6 собрана за 82.98 сек
🔎 Загружаем страницу 7: https://kinogo.org/serials/page/7/
✅ Страница 7 собрана за 6.08 сек
🔎 Загружаем страницу 8: https://kinogo.org/serials/page/8/
✅ Страница 8 собрана за 8.24 сек
🔎 Загружаем страницу 9: https://kinogo.org/serials/page/9/
✅ Страница 9 собрана за 6.74 сек
🔎 Загружаем страницу 10: https://kinogo.org/serials/page/10/
✅ Страница 10 собрана за 5.99 сек
🎬 Основной парсинг завершён. Данные сохранены в movies_dataset.csv и в mo

KeyboardInterrupt: 

### Оптимизированное предложение

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
from tqdm import tqdm  # добавляем tqdm для прогресс-бара


# --- Настройки Selenium ---
def get_browser():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    service = Service(executable_path="/usr/bin/chromedriver")
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 YaBrowser/25.6.0.0 Safari/537.36"
    })
    return browser

browser = get_browser()

# --- Основные настройки ---
url_main = 'https://kinogo.org/serials/'
page_min, page_max = 1, 550
my_range = range(page_min, page_max+1)
save_every = 5   # сохранять каждые N страниц

final_dict = {k: [] for k in [
    'title','out_year','genres','country','duration','description',
    'url_movie','image_movie','rating_votes','film_rating_kp','film_rating_imdb'
]}

short_descriptions = []  # для подстраховки, если нет полного описания

# --- Парсим список страниц ---
for page in my_range:
    start_time = time.time()
    url = f"{url_main}page/{page}/"
    print(f"🔎 Загружаем страницу {page}: {url}")

    browser.get(url)
    WebDriverWait(browser, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "shortStory"))
    )
    soup = BeautifulSoup(browser.page_source, "lxml")
    blocks = soup.find_all(class_="shortStory")

    for block in blocks:
        # --- Название ---
        title = block.find("h2").get_text(strip=True).split('(')[0].strip() if block.find("h2") else None
        # --- Год ---
        year_tag = block.find('b', string='Год выпуска:')
        year = ", ".join([tag.get_text() for tag in year_tag.find_next_siblings('a')]) if year_tag else None
        # --- URL фильма ---
        url_movie = block.find("a").get("href") if block.find("a") else None
        # --- Постер ---
        img_tag = block.find("div", class_="sPoster").find("img").get('data-src') if block.find("div", class_="sPoster") else None
        img_url = f'https://kinogo.org{img_tag}' if img_tag else None
        # --- Жанры ---
        genre_tag = block.find('b', string='Жанр:')
        genres = [g.get_text() for g in genre_tag.find_next_siblings('a')] if genre_tag else None
        # --- Страна ---
        country_tag = block.find('b', string='Страна:')
        country = [c.get_text() for c in country_tag.find_next_siblings('a')] if country_tag else None
        # --- Продолжительность ---
        duration_tag = block.find('b', string='Продолжительность:')
        duration = duration_tag.next_sibling.strip() if duration_tag else None
        # --- Краткое описание ---
        desc_tag = block.find("p", class_="excerpt")
        short_desc = desc_tag.get_text(strip=True) if desc_tag else None
        # --- Рейтинг сайта ---
        rating_tag = block.find("span", class_="ratingStats")
        rating = rating_tag.get_text(strip=True).split()[0] if rating_tag else None
        # --- Рейтинг Кинопоиск ---
        kp_tag = block.find("span", class_="fullKP")
        kp = kp_tag.get_text(strip=True).split(':')[1] if kp_tag else None
        # --- Рейтинг IMDb ---
        imdb_tag = block.find("span", class_="fQuality")
        imdb = imdb_tag.get_text(strip=True).split(':')[1] if imdb_tag else None

        # --- Записываем ---
        final_dict['title'].append(title)
        final_dict['out_year'].append(year)
        final_dict['url_movie'].append(url_movie)
        final_dict['image_movie'].append(img_url)
        final_dict['genres'].append(genres)
        final_dict['country'].append(country)
        final_dict['duration'].append(duration)
        final_dict['description'].append(short_desc)  # временно ставим краткое
        final_dict['rating_votes'].append(rating)
        final_dict['film_rating_kp'].append(kp)
        final_dict['film_rating_imdb'].append(imdb)

        short_descriptions.append(short_desc)

    print(f"✅ Страница {page} собрана за {time.time()-start_time:.2f} сек")

    # --- частое сохранение ---
    if page % save_every == 0:
        df = pd.DataFrame(final_dict)
        df.to_csv('movies_dataset_partial_5500.csv', index=False)
        with open('movies_dataset_partial_5500.json', 'w', encoding='utf-8') as f:
            json.dump(final_dict, f, ensure_ascii=False, indent=4)
        print(f"💾 Прогресс сохранён после страницы {page}")

# --- Функция для скачивания полного описания ---
def get_full_description(idx, url, short_desc):
    try:
        b = get_browser()
        b.get(url)
        WebDriverWait(b, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "filmDescription"))
        )
        movie_soup = BeautifulSoup(b.page_source, "lxml")
        desc_block = movie_soup.find("div", class_="filmDescription")
        b.quit()
        return idx, desc_block.get_text(strip=True) if desc_block else short_desc
    except Exception as e:
        print(f"⚠ Ошибка при загрузке {url}: {e}")
        return idx, short_desc

# --- Парсим полные описания в потоках ---
print("🔎 Загружаем полные описания (многопоточно)...")
full_descriptions = [None] * len(final_dict['url_movie'])

with ThreadPoolExecutor(max_workers=5) as executor:  # число потоков подбирай под железо
    futures = []
    for idx, url in enumerate(final_dict['url_movie']):
        short_desc = short_descriptions[idx] if idx < len(short_descriptions) else None
        if url:
            futures.append(executor.submit(get_full_description, idx, url, short_desc))
        else:
            full_descriptions[idx] = short_desc

    # прогресс-бар
    for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг описаний"):
        idx, desc = future.result()
        full_descriptions[idx] = desc

# --- Заменяем краткие на полные ---
final_dict['description'] = full_descriptions

# --- Финальное сохранение ---
df = pd.DataFrame(final_dict)
full_name_csv = 'movies_dataset_full_5500.csv'
full_name_json = 'movies_dataset_full_5500.json'
df.to_csv(full_name_csv, index=False)
with open(full_name_json, 'w', encoding='utf-8') as f:
    json.dump(final_dict, f, ensure_ascii=False, indent=4)

print(f"🎬 Парсинг завершён. Данные сохранены в {full_name_csv} и {full_name_json}")
print(f'Количество фильмов в базе: {df.shape[0]}')

browser.quit()

🔎 Загружаем страницу 1: https://kinogo.org/serials/page/1/
✅ Страница 1 собрана за 3.27 сек
🔎 Загружаем страницу 2: https://kinogo.org/serials/page/2/
✅ Страница 2 собрана за 1.61 сек
🔎 Загружаем страницу 3: https://kinogo.org/serials/page/3/
✅ Страница 3 собрана за 1.11 сек
🔎 Загружаем страницу 4: https://kinogo.org/serials/page/4/
✅ Страница 4 собрана за 0.97 сек
🔎 Загружаем страницу 5: https://kinogo.org/serials/page/5/
✅ Страница 5 собрана за 1.05 сек
💾 Прогресс сохранён после страницы 5
🔎 Загружаем страницу 6: https://kinogo.org/serials/page/6/
✅ Страница 6 собрана за 1.09 сек
🔎 Загружаем страницу 7: https://kinogo.org/serials/page/7/
✅ Страница 7 собрана за 1.38 сек
🔎 Загружаем страницу 8: https://kinogo.org/serials/page/8/
✅ Страница 8 собрана за 1.06 сек
🔎 Загружаем страницу 9: https://kinogo.org/serials/page/9/
✅ Страница 9 собрана за 1.22 сек
🔎 Загружаем страницу 10: https://kinogo.org/serials/page/10/
✅ Страница 10 собрана за 1.31 сек
💾 Прогресс сохранён после страницы 10
🔎 

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=58103): Read timed out. (read timeout=120)

In [168]:
pd.read_csv('movies_dataset_full_new.csv')

,title,out_year,genres,country,duration,description,url_movie,image_movie,rating_votes,film_rating_kp,film_rating_imdb
0,Бахар,2024,"['драма', 'комедия', 'мелодрама']",['Турция'],2 ч 12 мин,«Бахар» – это захватывающая мелодрама 2024 год...,https://kinogo.org/109108-bahar.html,https://kinogo.org/uploads/mini/short/948/20ee...,3.7/5,7.9,7.2
1,ПИН-код,2025,"['детектив', 'драма']",['Россия'],49 мин,"Максим Захаров, майор полиции, сталкивается с ...",https://kinogo.org/126489-pin-kod.html,https://kinogo.org/uploads/mini/short/09c/9fe2...,4.2/5,Неизвестно,Неизвестно
2,Рейс 314,2025,['комедия'],['Россия'],23 мин,"Иван Никогда, бывший чиновник с коррупционным ...",https://kinogo.org/126326-rejs-314.html,https://kinogo.org/uploads/mini/short/3b0/5618...,3.2/5,Неизвестно,Неизвестно
3,Виноделы,2025,"['комедия', 'мелодрама']",['Россия'],48 мин,Шашлычник Паша из Челябинска вместе с отцом и ...,https://kinogo.org/126323-vinodely.html,https://kinogo.org/uploads/mini/short/13e/6ddd...,3.1/5,7.9,Неизвестно
4,Планетяне,2025,"['комедия', 'фантастика']",['Россия'],23 мин,Двое представителей пришельцев прибыли на Земл...,https://kinogo.org/126325-planetjane.html,https://kinogo.org/uploads/mini/short/732/1e42...,3.4/5,7.6,Неизвестно
...,...,...,...,...,...,...,...,...,...,...,...
95,Москва слезам не верит. Всё только начинается,2025,"['драма', 'мелодрама']",['Россия'],52 мин,В начале 2000-х годов Ксюша отправляется на за...,https://kinogo.org/126000-moskva-slezam-ne-ver...,https://kinogo.org/uploads/mini/short/882/900b...,3.6/5,5.9,Неизвестно
96,Тайная история принцессы Тайпин,2012,"['боевик', 'драма', 'история']",['Китай'],Неизвестно,"Это историческая драма 2012 года, которая осно...",https://kinogo.org/118951-tajnaja-istorija-pri...,https://kinogo.org/uploads/mini/short/fec/4396...,4/5,Неизвестно,7.1
97,Непослушные,2025,"['детектив', 'драма', 'триллер']","['Великобритания', 'Канада', 'США']",46 мин,"Офицер полиции Алекс Демпси, работающий в небо...",https://kinogo.org/126400-neposlushnye.html,https://kinogo.org/uploads/mini/short/a09/869a...,4/5,Неизвестно,7.2
98,Дом Гиннесса,2025,"['биография', 'драма', 'история']",['Великобритания'],53 мин,В центре сюжета данного телесериала — история ...,https://kinogo.org/125964-dom-ginnessa.html,https://kinogo.org/uploads/mini/short/b11/495f...,3.4/5,Неизвестно,Неизвестно


### Оптимизированное №2

In [3]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
from tqdm import tqdm  # добавляем tqdm для прогресс-бара

# --- Настройки Selenium ---
def get_browser():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    service = Service(executable_path="/usr/bin/chromedriver")
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 YaBrowser/25.6.0.0 Safari/537.36"
    })
    return browser

browser = get_browser()

# --- Основные настройки ---
url_main = 'https://kinogo.org/serials/'
page_min, page_max = 1, 550
my_range = range(page_min, page_max+1)
save_every = 5   # сохранять каждые N страниц
restart_every = 50  # перезапуск браузера каждые N страниц

final_dict = {k: [] for k in [
    'title','out_year','genres','country','duration','description',
    'url_movie','image_movie','rating_votes','film_rating_kp','film_rating_imdb'
]}

short_descriptions = []  # для подстраховки, если нет полного описания
skipped_pages = []       # список пропущенных страниц

# --- Парсим список страниц ---
for page in my_range:
    start_time = time.time()
    url = f"{url_main}page/{page}/"
    print(f"🔎 Загружаем страницу {page}: {url}")

    try:
        # перезапуск браузера каждые restart_every страниц
        if page % restart_every == 0 and page != page_min:
            browser.quit()
            browser = get_browser()
            print(f"♻ Перезапущен браузер на странице {page}")

        browser.set_page_load_timeout(30)
        browser.get(url)

        WebDriverWait(browser, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "shortStory"))
        )
        soup = BeautifulSoup(browser.page_source, "lxml")
        blocks = soup.find_all(class_="shortStory")

        for block in blocks:
            # --- Название ---
            title = block.find("h2").get_text(strip=True).split('(')[0].strip() if block.find("h2") else None
            # --- Год ---
            year_tag = block.find('b', string='Год выпуска:')
            year = ", ".join([tag.get_text() for tag in year_tag.find_next_siblings('a')]) if year_tag else None
            # --- URL фильма ---
            url_movie = block.find("a").get("href") if block.find("a") else None
            # --- Постер ---
            img_tag = block.find("div", class_="sPoster").find("img").get('data-src') if block.find("div", class_="sPoster") else None
            img_url = f'https://kinogo.org{img_tag}' if img_tag else None
            # --- Жанры ---
            genre_tag = block.find('b', string='Жанр:')
            genres = [g.get_text() for g in genre_tag.find_next_siblings('a')] if genre_tag else None
            # --- Страна ---
            country_tag = block.find('b', string='Страна:')
            country = [c.get_text() for c in country_tag.find_next_siblings('a')] if country_tag else None
            # --- Продолжительность ---
            duration_tag = block.find('b', string='Продолжительность:')
            duration = duration_tag.next_sibling.strip() if duration_tag else None
            # --- Краткое описание ---
            desc_tag = block.find("p", class_="excerpt")
            short_desc = desc_tag.get_text(strip=True) if desc_tag else None
            # --- Рейтинг сайта ---
            rating_tag = block.find("span", class_="ratingStats")
            rating = rating_tag.get_text(strip=True).split()[0] if rating_tag else None
            # --- Рейтинг Кинопоиск ---
            kp_tag = block.find("span", class_="fullKP")
            kp = kp_tag.get_text(strip=True).split(':')[1] if kp_tag else None
            # --- Рейтинг IMDb ---
            imdb_tag = block.find("span", class_="fQuality")
            imdb = imdb_tag.get_text(strip=True).split(':')[1] if imdb_tag else None

            # --- Записываем ---
            final_dict['title'].append(title)
            final_dict['out_year'].append(year)
            final_dict['url_movie'].append(url_movie)
            final_dict['image_movie'].append(img_url)
            final_dict['genres'].append(genres)
            final_dict['country'].append(country)
            final_dict['duration'].append(duration)
            final_dict['description'].append(short_desc)  # временно ставим краткое
            final_dict['rating_votes'].append(rating)
            final_dict['film_rating_kp'].append(kp)
            final_dict['film_rating_imdb'].append(imdb)

            short_descriptions.append(short_desc)

        print(f"✅ Страница {page} собрана за {time.time()-start_time:.2f} сек")

    except Exception as e:
        print(f"⚠ Пропускаем страницу {page}, ошибка: {e}")
        skipped_pages.append(page)
        continue

    # --- частое сохранение ---
    if page % save_every == 0:
        df = pd.DataFrame(final_dict)
        df.to_csv('movies_dataset_partial_5500.csv', index=False)
        with open('movies_dataset_partial_5500.json', 'w', encoding='utf-8') as f:
            json.dump(final_dict, f, ensure_ascii=False, indent=4)
        print(f"💾 Прогресс сохранён после страницы {page}")

# --- Итог по пропущенным страницам ---
print(f"\n🎬 Парсинг списка завершён. Пропущено страниц: {len(skipped_pages)}")
if skipped_pages:
    print("Список пропущенных:", skipped_pages)

# --- Функция для скачивания полного описания ---
def get_full_description(idx, url, short_desc):
    try:
        b = get_browser()
        b.get(url)
        WebDriverWait(b, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "filmDescription"))
        )
        movie_soup = BeautifulSoup(b.page_source, "lxml")
        desc_block = movie_soup.find("div", class_="filmDescription")
        b.quit()
        return idx, desc_block.get_text(strip=True) if desc_block else short_desc
    except Exception as e:
        print(f"⚠ Ошибка при загрузке {url}: {e}")
        return idx, short_desc

# --- Парсим полные описания в потоках ---
print("🔎 Загружаем полные описания (многопоточно)...")
full_descriptions = [None] * len(final_dict['url_movie'])
skipped_descriptions = []  # список пропущенных фильмов

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for idx, url in enumerate(final_dict['url_movie']):
        short_desc = short_descriptions[idx] if idx < len(short_descriptions) else None
        if url:
            futures.append(executor.submit(get_full_description, idx, url, short_desc))
        else:
            full_descriptions[idx] = short_desc

    for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг описаний"):
        idx, desc = future.result()
        full_descriptions[idx] = desc

# --- Заменяем краткие на полные ---
final_dict['description'] = full_descriptions

# --- Финальное сохранение ---
df = pd.DataFrame(final_dict)
full_name_csv = 'movies_dataset_full_5500.csv'
full_name_json = 'movies_dataset_full_5500.json'
df.to_csv(full_name_csv, index=False)
with open(full_name_json, 'w', encoding='utf-8') as f:
    json.dump(final_dict, f, ensure_ascii=False, indent=4)

print(f"🎬 Парсинг завершён. Данные сохранены в {full_name_csv} и {full_name_json}")
print(f'Количество фильмов в базе: {df.shape[0]}')

browser.quit()


🔎 Загружаем страницу 1: https://kinogo.org/serials/page/1/
✅ Страница 1 собрана за 3.33 сек
🔎 Загружаем страницу 2: https://kinogo.org/serials/page/2/
✅ Страница 2 собрана за 0.87 сек
🔎 Загружаем страницу 3: https://kinogo.org/serials/page/3/
✅ Страница 3 собрана за 0.88 сек
🔎 Загружаем страницу 4: https://kinogo.org/serials/page/4/
✅ Страница 4 собрана за 0.94 сек
🔎 Загружаем страницу 5: https://kinogo.org/serials/page/5/
✅ Страница 5 собрана за 1.81 сек
💾 Прогресс сохранён после страницы 5
🔎 Загружаем страницу 6: https://kinogo.org/serials/page/6/
✅ Страница 6 собрана за 2.35 сек
🔎 Загружаем страницу 7: https://kinogo.org/serials/page/7/
✅ Страница 7 собрана за 0.98 сек
🔎 Загружаем страницу 8: https://kinogo.org/serials/page/8/
✅ Страница 8 собрана за 0.95 сек
🔎 Загружаем страницу 9: https://kinogo.org/serials/page/9/
✅ Страница 9 собрана за 0.86 сек
🔎 Загружаем страницу 10: https://kinogo.org/serials/page/10/
✅ Страница 10 собрана за 0.95 сек
💾 Прогресс сохранён после страницы 10
🔎 

Парсинг описаний:   1%|          | 47/5500 [02:21<7:21:21,  4.86s/it]

⚠ Ошибка при загрузке https://kinogo.org/126458-chasovye.html: Message: 
Stacktrace:
#0 0x5799ec0a296a <unknown>
#1 0x5799ebb38e60 <unknown>
#2 0x5799ebb89300 <unknown>
#3 0x5799ebb894f1 <unknown>
#4 0x5799ebbd78f4 <unknown>
#5 0x5799ebbaf15d <unknown>
#6 0x5799ebbd4cc5 <unknown>
#7 0x5799ebbaef03 <unknown>
#8 0x5799ebb7ba5a <unknown>
#9 0x5799ebb7c6d1 <unknown>
#10 0x5799ec067eff <unknown>
#11 0x5799ec06bf08 <unknown>
#12 0x5799ec04eddc <unknown>
#13 0x5799ec06c6ae <unknown>
#14 0x5799ec03629e <unknown>
#15 0x5799ec090238 <unknown>
#16 0x5799ec090448 <unknown>
#17 0x5799ec0a156c <unknown>
#18 0x7d75eb494ac3 <unknown>



Парсинг описаний:   1%|          | 63/5500 [03:11<4:49:14,  3.19s/it]

⚠ Ошибка при загрузке https://kinogo.org/58834-kljukvennyj-scherbet.html: Message: 
Stacktrace:
#0 0x56939822896a <unknown>
#1 0x569397cbee60 <unknown>
#2 0x569397d0f300 <unknown>
#3 0x569397d0f4f1 <unknown>
#4 0x569397d5d8f4 <unknown>
#5 0x569397d3515d <unknown>
#6 0x569397d5acc5 <unknown>
#7 0x569397d34f03 <unknown>
#8 0x569397d01a5a <unknown>
#9 0x569397d026d1 <unknown>
#10 0x5693981edeff <unknown>
#11 0x5693981f1f08 <unknown>
#12 0x5693981d4ddc <unknown>
#13 0x5693981f26ae <unknown>
#14 0x5693981bc29e <unknown>
#15 0x569398216238 <unknown>
#16 0x569398216448 <unknown>
#17 0x56939822756c <unknown>
#18 0x7965b8494ac3 <unknown>



...

Парсинг описаний: 100%|█████████▉| 5496/5500 [4:01:26<00:59, 14.89s/it]

⚠ Ошибка при загрузке https://kinogo.org/101766-solnechnyj-sezon.html: Message: Can not connect to the Service /usr/bin/chromedriver
⚠ Ошибка при загрузке https://kinogo.org/101689-kim-su-ro.html: Message: Can not connect to the Service /usr/bin/chromedriver
⚠ Ошибка при загрузке https://kinogo.org/98784-luchshaja-oshibka.html: Message: Can not connect to the Service /usr/bin/chromedriver

⚠ Ошибка при загрузке https://kinogo.org/101730-hudozhnik-dush.html: Message: Can not connect to the Service /usr/bin/chromedriver





Парсинг описаний: 100%|██████████| 5500/5500 [4:01:27<00:00,  2.63s/it]

⚠ Ошибка при загрузке https://kinogo.org/101719-mertvye-dengi.html: Message: Can not connect to the Service /usr/bin/chromedriver



🎬 Парсинг завершён. Данные сохранены в movies_dataset_full_5500.csv и movies_dataset_full_5500.json
Количество фильмов в базе: 5500
